In [ ]:
pip install ultralytics opencv-python numpy

In [ ]:
# generating images for training of our model

from PIL import Image, ImageDraw, ImageFont
import random

def generate_digit_with_decimal():
    # Generate a random number with decimal
    number = str(random.randint(0, 9)) + "." + str(random.randint(0, 99))
    image = Image.new('RGB', (100, 30), (255, 255, 255))
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    draw.text((10, 5), number, font=font, fill=(0, 0, 0))
    image.save(fr"C:\Users\athar\OneDrive\Desktop\images/{number}.png")

# Generate multiple images
for _ in range(1000000000):
    generate_digit_with_decimal()

In [ ]:
## now sizing the images accordingly for the 

import os
import cv2
import easyocr
import numpy as np

# Function to convert bounding box coordinates to YOLO format
# Function to convert bounding box coordinates to YOLO format
def convert_to_yolo_format(image_width, image_height, bbox):
    # Extract x_min, y_min, x_max, y_max from the bounding box
    x_min, y_min = bbox[0]
    x_max, y_max = bbox[2]
    
    # Calculate the center_x, center_y, width, and height in YOLO format
    center_x = (x_min + x_max) / 2.0 / image_width
    center_y = (y_min + y_max) / 2.0 / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    
    return (center_x, center_y, width, height)


# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Path to the folder containing images
image_folder = r"C:\Users\athar\OneDrive\Desktop\images"
output_folder = r"C:\Users\athar\OneDrive\Desktop\nww"

# Create output folder if not exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the folder
for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
        # Load image
        image_path = os.path.join(image_folder, image_filename)
        image = cv2.imread(image_path)
        image_height, image_width, _ = image.shape
        
        # Detect text using EasyOCR
        result = reader.readtext(image_path)
        
        # Initialize list for bounding box annotations in YOLO format
        yolo_annotations = []

        # Process each detected text box
        for detection in result:
            text = detection[1]  # The text (digit/decimal point)The bounding box coordinates
            cleaned_text = text.replace(" ", "")  # Remove spaces
            # Ignore non-digit and non-decimal characters (optional)
            if cleaned_text.replace('.', '').isdigit(): 
                # Convert bounding box to YOLO format
                 yolo_bbox = convert_to_yolo_format(image_width, image_height, detection[0])
                
                # Determine class ID (0-9 for digits, 10 for decimal point)
                 class_id = 10 if '.' in cleaned_text else int(cleaned_text)
                  
                # Append annotation (class_id + bounding box)
                 yolo_annotations.append(f"{class_id} {' '.join(map(str, yolo_bbox))}")
        
        # Save annotations to YOLO format text file
        label_filename = os.path.splitext(image_filename)[0] + '.txt'
        label_path = os.path.join(output_folder, label_filename)
        
        with open(label_path, 'w') as label_file:
            for annotation in yolo_annotations:
                label_file.write(annotation + '\n')

        print(f"Processed {image_filename} and saved annotations to {label_filename}")

In [ ]:
##saving the yaml file for the same to train the model on YOLO
import os

# Define dataset paths
dataset_path = r"C:\Users\athar\OneDrive\Desktop\data"
train_images = os.path.join(dataset_path, r"C:\Users\athar\OneDrive\Desktop\images")

# Define class names (digits 0-9 and decimal point)
classes = [str(i) for i in range(10)] + ["decimal"]

# Create dataset.yaml content (without validation set)
yaml_content = f"""train: {train_images}
val: {train_images}  # Using train set as val since val doesn't exist

nc: {len(classes)}  # Number of classes
names: {classes}
"""

# Save to dataset.yaml
yaml_path = os.path.join(dataset_path, r"dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)


In [1]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolov8n.pt')

# Train the model using the automatically created dataset.yaml
model.train(data=r"C:\Users\athar\OneDrive\Desktop\data\dataset.yaml", epochs=1000, imgsz=640, batch=16, optimizer='Adam', lr0=0.001, name='handwritten_number_detector')


New https://pypi.org/project/ultralytics/8.3.70 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.222 🚀 Python-3.11.5 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\athar\OneDrive\Desktop\data\dataset.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=handwritten_number_detector5, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

train: Scanning C:\Users\athar\OneDrive\Desktop\labels.cache... 636 images, 563 backgrounds, 37 corrupt: 100%|█████████

train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\0.12.png: ignoring corrupt image/label: Label class 12 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\0.17.png: ignoring corrupt image/label: Label class 17 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.16.png: ignoring corrupt image/label: Label class 116 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.18.png: ignoring corrupt image/label: Label class 118 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.51.png: ignoring corrupt image/label: Label class 151 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.81.png: ignoring corrupt image/label: Label class 181 exceeds d


val: Scanning C:\Users\athar\OneDrive\Desktop\labels.cache... 636 images, 563 backgrounds, 37 corrupt: 100%|██████████|

train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\0.12.png: ignoring corrupt image/label: Label class 12 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\0.17.png: ignoring corrupt image/label: Label class 17 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.16.png: ignoring corrupt image/label: Label class 116 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.18.png: ignoring corrupt image/label: Label class 118 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.51.png: ignoring corrupt image/label: Label class 151 exceeds dataset class count 11. Possible class labels are 0-10
train: WARNING ⚠️ C:\Users\athar\OneDrive\Desktop\images\1.81.png: ignoring corrupt image/label: Label class 181 exceeds d

...

                   all        599         36      0.969      0.858      0.963      0.878



100 epochs completed in 0.243 hours.
Optimizer stripped from runs\detect\handwritten_number_detector5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\handwritten_number_detector5\weights\best.pt, 6.2MB

Validating runs\detect\handwritten_number_detector5\weights\best.pt...
Ultralytics YOLOv8.0.222 🚀 Python-3.11.5 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3007793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:01


                   all        599         36      0.932      0.833      0.951      0.924
               decimal        599         36      0.932      0.833      0.951      0.924
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\handwritten_number_detector5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([10])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002B2BC2C4310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048

In [ ]:
import csv
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO('runs/detect/handwritten_number_detector5/weights/best.pt')

# Run inference on the image
image_path = r"C:\Users\athar\Downloads\i1.JPG"
results = model.predict(image_path, conf=0.25, half=False)

# Extract the predictions from the results
predictions = results[0].boxes
labels = predictions.cls  # Predicted classes (digit labels)
coordinates = predictions.xyxy  # Bounding box coordinates for each detected digit

# Convert labels to integers and prepare a list of detected digits
detected_digits = [int(label) for label in labels]
print("Detected Digits:", detected_digits)

# Save the detected digits and image name to a CSV file
csv_filename = "detected_digits.csv"
with open(csv_filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([image_path, ', '.join(map(str, detected_digits))])

print(f"Detected digits for {image_path} saved to {csv_filename}")

